In [30]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Kaan\AppData\Local\Temp\ipykernel_30000\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [32]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\DSML Bootcamp\Week 7\lab-natural-language-processing\data\kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [48]:
X = data['text']
Y = data['label']
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [49]:
y_train

521    Dear Sir=2C I wish you go through this offer t...
737    To take your mind off the Balkans for a second...
740                         Pls keep the updates coming!
660    </STRONG><STRONG>CHRIST BETHEL HOSPITAL<BR>11 ...
411    sbwhoeopFriday February 5 2010 7:11 AMHRe: Bra...
                             ...                        
408    Sorry yes exactlyWe have shy tomorrow at 10am ...
332    DEAR=2CGOOD DAY=2EI KNOW THIS MESSAGE WILL COM...
208                                                  FYI
613    Greetings Dear Friend Please Permit me to cont...
78     No in car on way to airport. Can you talk? Cal...
Name: text, Length: 200, dtype: object

## Data Preprocessing

In [34]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [35]:
import re
import pandas as pd

# Diese Funktion haben wir zuvor erstellt
def clean_html_code(raw_html):
    # Entfernen von <script>-Tags und ihrem Inhalt
    clean_text = re.sub(r'<script\b[^>]*>.*?</script>', '', raw_html, flags=re.DOTALL)
    # Entfernen von <style>-Tags und ihrem Inhalt
    clean_text = re.sub(r'<style\b[^>]*>.*?</style>', '', clean_text, flags=re.DOTALL)
    # Entfernen von HTML-Kommentaren
    clean_text = re.sub(r'', '', clean_text, flags=re.DOTALL)
    # Entfernen aller verbleibenden Tags
    clean_text = re.sub(r'<[^>]+>', '', clean_text)
    # Entfernen von überflüssigen Leerzeichen
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

# Annahme: X_train ist bereits eine Pandas Series oder ein DataFrame-Ausschnitt
# Wenden Sie die Funktion auf jeden Text-Eintrag in X_train an
X_train = X_train.apply(clean_html_code)

# Zeigen Sie die ersten Einträge von X_train, um die Ergebnisse zu sehen
print(X_train.head())

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836    BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
Name: text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [36]:
import re

def clean_text(text):
    # 1. Konvertieren in Kleinbuchstaben
    text = text.lower()
    
    # 2. Entfernen von "b'" am Anfang (typisch für Byte-Strings)
    text = re.sub(r'^b\'|"', '', text)
    
    # 3. Entfernen von Zahlen
    text = re.sub(r'\d+', '', text)
    
    # 4. Entfernen aller Sonderzeichen
    # Nur Buchstaben und Leerzeichen behalten
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 5. Entfernen aller einzelnen Buchstaben (auch am Anfang)
    # Das entfernt Wörter mit nur einem Buchstaben. Der Code entfernt auch einzelne Buchstaben am Anfang und am Ende.
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # 6. Ersetzen von mehreren Leerzeichen durch ein einzelnes Leerzeichen
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Wende die Funktion auf dein X_train-Set an
# Dadurch wird jeder Eintrag im X_train-Set bereinigt
X_train_cleaned = X_train.apply(clean_text)

# Zeige die ersten paar bereinigten Einträge, um die Ergebnisse zu überprüfen
print(X_train_cleaned.head())

29     regards mr nelson smithkindly reply me on my p...
535    i have not been able to reach oscar this am we...
695    huma abedin bim checking with pat on the will ...
557    i can have it announced here on monday cant today
836    bank of africaagence san pedro bp san pedro co...
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    
    words = word_tokenize(text)
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    return " ".join(filtered_words)


X_train_final = X_train_cleaned.apply(remove_stopwords)

print(X_train_final.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kaan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kaan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


29     regards mr nelson smithkindly reply private em...
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [38]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Laden Sie die notwendigen Ressourcen herunter (einmalig ausführen)
nltk.download('wordnet')

# Initialisieren Sie den Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Zerlegt den Text in Wörter
    words = word_tokenize(text)
    
    # Lemmatisiert jedes Wort
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Fügt die Wörter wieder zu einem String zusammen
    return " ".join(lemmatized_words)

# Wenden Sie die Lemmatisierung auf Ihr bereinigtes X_train-Set an
# Nehmen Sie an, dass X_train_final aus dem vorherigen Schritt das bereinigte Set ist
X_train_final = X_train_final.apply(lemmatize_text)

# Zeigen Sie die ersten Einträge, um die Ergebnisse zu überprüfen
print(X_train_final.head())

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kaan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


29     regard mr nelson smithkindly reply private ema...
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [50]:
from collections import Counter
ham_messages = data[data['label'] == 0]['text']
spam_messages = data[data['label'] == 1]['text']
ham_words = [word for message in ham_messages for word in message.split()]
spam_words = [word for message in spam_messages for word in message.split()]
ham_top = Counter(ham_words).most_common(10)
spam_top = Counter(spam_words).most_common(10)
print("Top 10 words in HAM messages:")
print(ham_top)
print("\nTop 10 words in SPAM messages:")
print(spam_top)

Top 10 words in HAM messages:
[('the', 1593), ('to', 1039), ('of', 782), ('and', 781), ('a', 591), ('in', 540), ('that', 371), ('is', 353), ('for', 344), ('on', 283)]

Top 10 words in SPAM messages:
[('the', 5676), ('to', 4688), ('of', 4118), ('and', 3234), ('in', 2672), ('I', 2633), ('you', 2273), ('this', 2010), ('a', 1939), ('for', 1685)]


In [40]:
import pandas as pd
from collections import Counter

# Zuerst erstellen wir einen temporären DataFrame, um die bereinigten Texte
# und ihre Labels (y_train) zu verknüpfen.
combined_df = pd.DataFrame({'text': X_train_final, 'label': y_train})

# Filtern Sie die Daten in zwei separate Kategorien (ham und spam)
# Verwenden Sie nun die String-Werte '0' und '1'
ham_text = combined_df[combined_df['label'] == '0']['text']
spam_text = combined_df[combined_df['label'] == '1']['text']

# Funktion, die die häufigsten Wörter zählt
def get_top_words(text_series, num_words=10):
    all_text = " ".join(text_series.values)
    words = all_text.split()
    word_counts = Counter(words)
    return word_counts.most_common(num_words)

# Top 10 Wörter in Ham-Nachrichten finden
top_10_ham = get_top_words(ham_text)

# Top 10 Wörter in Spam-Nachrichten finden
top_10_spam = get_top_words(spam_text)

print("Die 10 häufigsten Wörter in Ham-Nachrichten (Label '0'):")
print(top_10_ham)

print("\nDie 10 häufigsten Wörter in Spam-Nachrichten (Label '1'):")
print(top_10_spam)

Die 10 häufigsten Wörter in Ham-Nachrichten (Label '0'):
[]

Die 10 häufigsten Wörter in Spam-Nachrichten (Label '1'):
[]


## Extra features

In [52]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

combined_df['money_mark'] = combined_df['preprocessed_text'].str.contains(money_simbol_list)*1
combined_df['suspicious_words'] = combined_df['preprocessed_text'].str.contains(suspicious_words)*1
combined_df['text_len'] = combined_df['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

combined_df.head()

KeyError: 'preprocessed_text'

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code